In [47]:
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
from fastlite import *
import fasthtml.components as fh
import httpx


# Fast HTML / Webdev fundamentals

In [48]:
daisy_hdrs = (
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
    Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4')
)


In [49]:
app = FastHTML(hdrs=daisy_hdrs)
app


In [50]:
rt = app.route


In [51]:

def get_preview(app): return partial(HTMX, app=app, host="localhost", port=8000)
p = get_preview(app)
p

functools.partial(<function HTMX at 0x10b85dc70>, app=<fasthtml.core.FastHTML object at 0x10d31cb90>, host='localhost', port=8000)

In [52]:
p = get_preview(app)


In [53]:
c = P("Hello, World", cls='text-3xl')
c


```html
<p class="text-3xl">Hello, World</p>

```

In [54]:
p(c)


In [55]:
acc = Div(
        Div(
        Input(type='radio', name='my-accordion-3', checked='checked'),
        Div('How do I create an account?', cls='collapse-title font-semibold'),
        Div('Click the "Sign Up" button in the top right corner and follow the registration process.', cls='collapse-content text-sm'),
        cls='collapse collapse-plus bg-base-100 border border-base-300'
    ),
    Div(
        Input(type='radio', name='my-accordion-3'),
        Div('I forgot my password. What should I do?', cls='collapse-title font-semibold'),
        Div('Click on "Forgot Password" on the login page and follow the instructions sent to your email.', cls='collapse-content text-sm'),
        cls='collapse collapse-plus bg-base-100 border border-base-300'
    ),
    Div(
        Input(type='radio', name='my-accordion-3'),
        Div('How do I update my profile information?', cls='collapse-title font-semibold'),
        Div('Go to "My Account" settings and select "Edit Profile" to make changes.', cls='collapse-content text-sm'),
        cls='collapse collapse-plus bg-base-100 border border-base-300'
    )
)


In [56]:
p(acc)


In [57]:
c = Button('Default', cls="btn")
p(c)


In [58]:
def Button(*c, cls='', **kw):
    return fh.Button(*c, cls=f"btn {cls}", **kw)


In [59]:
c = Button('Default')
p(c)



In [60]:
@rt
def showbtn(): return c


In [61]:
@rt
def index():
    return Titled('Hello, Jupyter',
           P('Welcome to the FastHTML + Jupyter example'),
           Button('Click', hx_get='/click', hx_target='#dest'),
           Div(id='dest')
    )


In [ ]:
server = JupyUvi(app)


ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 8000): [errno 48] address already in use


In [ ]:
@rt
def click(): return P('You clicked me!')


In [ ]:
p('/showbtn')



In [ ]:
def Input(*c, cls='', **kw): 
    return fh.Input(*c, cls=f'input {cls}', **kw)


In [ ]:
@rt
def btnres(nm:str='Dad'): return f"Hi, {nm}"

In [ ]:
btnres("ben")

'Hi, ben'

TO avoid typing stuff we don't need again, we replace the Input nameaspace with a wrapper funnction that calls the input class with out preferred args

In [ ]:
p(Input(type='text', placeholder='Type here'))

In [ ]:
@rt
def testbtn():
    return Form(hx_post=btnres, hx_target='#dest')(
        Input(placeholder='Who do you want to greet?', id="nm"),
        Button('Greet'),
        P(id='dest')
    )

In [ ]:
p(testbtn)

# Executing Code cells

In [ ]:
from toolslm.shell import get_shell


In [ ]:
sh = get_shell()

In [ ]:
sh.run_cell('1+1')

<ExecutionResult object at 10d2f8890, execution_count=None error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 10d1f1b50, raw_cell="1+1" transformed_cell="1+1
" store_history=False silent=False shell_futures=True cell_id=None> result=2>

In [ ]:
@rt
def execcode(code:str): return P(f"{code}")

In [ ]:
def ex(c):
    res = sh.run_cell(c)
    return res.result if res.result is not None else res.stdout


In [ ]:
ex("""1 + 10""")

11

In [ ]:
ex("""print('Hello, World')""")

'Hello, World\n'

In [ ]:
@rt
def runcode():
    return Form(hx_post=execcode, hx_target='#dest', hx_swap='afterbegin')(
        Input(placeholder='Code', id="code"),
        Button('Run code'),
        Div(id='dest')
    )

# Building solveit in solveit

In [ ]:
def mk_comp(comp, dflt_cls):
    def _f(*args, cls='', **kwargs): return comp(*args, cls=f'{dflt_cls} {cls}', **kwargs)
    return _f


In [ ]:
mk_comp('Button', 'btn')
mk_comp('Input', 'input')

<function __main__.mk_comp.<locals>._f(*args, cls='', **kwargs)>

In [ ]:
p(Button('Test button', cls='btn-primary'))

In [ ]:
p(Input(type="text", placeholder="Type here"))

In [ ]:
@rt
def getinp(): return Input(type='text', placeholder='Type here')

In [ ]:
p(getinp)

In [ ]:
html_obj = p(getinp, host=app)
html_obj.data

In [ ]:
html_obj = p(your_route)
html_obj.data  # What's the actual HTML?

### TODO: Find out how to  pass attributes first to classes to see how it works.

### TODO: Learn more about htmx reference objects. All apart from select and push-to-url. Which aren't very useful

In particular HTMX swap and how its relevant to the above.



## Stop server 
Top stop the server run the below

In [ ]:
# server.stop()